In [1]:
import random as random
import Bio
import numpy as np
import sys
import os
import networkx as nx
import random as random
from tqdm import tqdm # loading bar
from utils_copy import linear_C, get_cost_2, get_sequence_string, parse_fasta_multiple, create_score_matrix, write_alignments_to_file, linear_backtrack, fill_graph,new_sp_approxi_combi
from utils_copy import convert_to_desired_format_nr_version, compute_cost, my_traversal_simply, extend_alignment_chaos, find_min_span_edges_testing, parse_fasta_multiple_remove_n
import timeit
from utils_copy import al_integrity_testt
from old_for_testing.sp_approx import sp_approx
from old_for_testing.utils import *
from functions_multiple_version import *

score_matrix={'a': {'a': 0, 'c': 5, 'g': 2, 't': 5}, 'c': {'a': 5, 'c': 0, 'g': 5, 't': 2}, 'g': {'a': 2, 'c': 5, 'g': 0, 't': 5}, 't': {'a': 5, 'c': 2, 'g': 5, 't': 0}}
gap_cost=5



In [2]:
random_sequences_collection=[]
for i in range(0,100):
    random_pile=[]
    for j in range (0,10):
        a_sequence=[]
        for k in range(0,random.randrange(5,10)):
            a_sequence.append(random.choice(['a','c','t','g']))
        random_pile.append(a_sequence)
    random_sequences_collection.append(random_pile)

print(random_sequences_collection)

[[['g', 'g', 'a', 'c', 'g', 'g', 'g', 'a', 'g'], ['c', 'a', 'g', 'a', 't', 'g'], ['a', 'c', 't', 't', 'a', 't'], ['g', 'c', 'c', 'g', 'g', 'a'], ['g', 'a', 't', 'g', 'g', 'c', 'a', 'a', 'a'], ['c', 't', 'a', 'a', 'a', 'c', 'g'], ['c', 'a', 'a', 'c', 'g', 'g', 'a', 'a'], ['g', 'c', 'g', 'g', 't'], ['a', 'c', 'c', 't', 'a', 'c', 'c', 'g'], ['g', 'a', 't', 'g', 't', 'g']], [['g', 't', 't', 't', 'c', 't', 'c', 'g', 't'], ['c', 'c', 'a', 'c', 't'], ['a', 't', 't', 'a', 't'], ['c', 'c', 'c', 't', 't', 'g', 'c', 'g', 'g'], ['a', 'g', 'c', 'c', 't', 't', 'g'], ['t', 'a', 'g', 'c', 'c', 'a', 't', 'c'], ['c', 'a', 't', 'g', 'g', 'a'], ['t', 'a', 'c', 'g', 'g', 'c', 'a'], ['t', 't', 't', 't', 't', 'g', 'a'], ['g', 't', 'g', 't', 'a', 'g', 'a', 't', 'g']], [['g', 'g', 'g', 'a', 'a', 'c', 'a', 'g', 't'], ['g', 't', 'a', 'a', 't'], ['a', 'a', 'g', 'c', 'c'], ['c', 'a', 'g', 'g', 'a', 'g'], ['a', 'a', 't', 'a', 'c', 'a', 'c', 'a', 'c'], ['g', 'c', 'a', 'g', 'c', 'a', 'a', 'g', 'c'], ['g', 'g', 't', '

In [3]:
costs_new=[]
costs_old=[]
for random_pile in random_sequences_collection:
    cost=new_assembly(random_pile,score_matrix,gap_cost)[4]
    costs_new.append(cost)
for random_pile in random_sequences_collection:
    cost=sp_approx(random_pile,score_matrix,gap_cost)[0]
    costs_old.append(cost)
print(costs_new)
print(costs_old)
costs_new=np.array(costs_new)
costs_old=np.array(costs_old)
differences=np.subtract(costs_new,costs_old)
print(differences)

Here comes the distance matrix produced by the alignments: 

[[ 0. 24. 30. 20. 16. 28. 14. 25. 29. 22.]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
Here comes the distance matrix produced by the alignments: 

[[ 0. 24. 30. 20. 16. 28. 14. 25. 29. 22.]
 [24.  0. 20. 19. 26.  9. 19. 17. 22. 12.]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
Here comes the d

SystemExit: 

/home/ana/mambaforge/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
count=np.count_nonzero(differences<0)
print("Your algo was better than the classic in "+ str(count)+"% of the time")

Your algo was better than the classic in 13% of the time
